In [2]:
# Dependencies
import numpy as np
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [3]:
engine = create_engine("sqlite:///hawaii.sqlite")

# get tables
inspector_morse = inspect(engine)
tables = inspector_morse.get_table_names()
for table in tables:
    print(table)
    
    columns = inspector_morse.get_columns(table)
    for column in columns:
        print(column)
    print()

measurement
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'date', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'prcp', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'tobs', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}

station
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'name', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'latitude', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoi

In [4]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [5]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [6]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
june_temps = session.query(Measurement).\
    filter(func.strftime("%m", Measurement.date) == "06").all()
june_temps[0:5] 

In [9]:
#  3. Convert the June temperatures to a list.
# tobs is 'temperature observations'
tobs = [x.tobs for x in june_temps]
tobs[0:5] 

[78.0, 76.0, 78.0, 76.0, 77.0]

In [12]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
# Make a connection to the SQL database
conn = engine.connect()

june_df = pd.DataFrame()
june_df["June Temps"] = tobs
june_df.head() 


,June Temps
0,78.0
1,76.0
2,78.0
3,76.0
4,77.0


In [13]:
query = """
        SELECT
            *
        from
            measurement m
        join station s on m.station = s.station
        WHERE
            date like '%-06-%';
        """

# Have a look at it all
df = pd.read_sql(query, conn)
df.head()

,id,station,date,prcp,tobs,id,station,name,latitude,longitude,elevation
0,134,USC00519397,2010-06-01,0.00,78.0,1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,135,USC00519397,2010-06-02,0.01,76.0,1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
2,136,USC00519397,2010-06-03,0.00,78.0,1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
3,137,USC00519397,2010-06-04,0.00,76.0,1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
4,138,USC00519397,2010-06-05,0.00,77.0,1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0


In [14]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
june_df.describe() 

,June Temps
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [18]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
december_temps = session.query(Measurement).\
    filter(func.strftime("%m", Measurement.date) == "12").all()
december_temps[0:5] 

In [19]:
# 7. Convert the December temperatures to a list.
tobs = [x.tobs for x in december_temps]
tobs[0:5] 

[76.0, 74.0, 74.0, 64.0, 64.0]

In [20]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
december_df = pd.DataFrame()
december_df["June Temps"] = tobs
december_df.head() 

,June Temps
0,76.0
1,74.0
2,74.0
3,64.0
4,64.0


In [22]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
december_df.describe()

,June Temps
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000
